# FiGS Examples Notebook
This notebook shows you how to convert a capture into a GSplat and then simulate a drone trajectory within it. The trajectory is flown by an MPC expert and can be viewed through a video output from the onboard camera of the drone.

Some useful settings for interactive work

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget

Importing the necessary libraries


In [ ]:
import figs.render.capture_generation as pg
import figs.visualize.generate_videos as gv

from figs.simulator import Simulator
from figs.control.vehicle_rate_mpc import VehicleRateMPC

#### 1. Define the capture (video name in gsplats/capture)

In [ ]:
# capture_name = "backroom"
capture_name = "button"

#### 2. Generate FiGS environment from capture

In [ ]:
pg.generate_gsplat(capture_name)

#### 3. Define the scene and course

In [ ]:
scene_name, course_name = "button", "button_prod"       # A short test trajectory where the drone approaches a button
# scene_name, course_name = "mid_gate", "traverse"        # Robustness trajecytory in the SOUS VIDE paper
# scene_name, course_name = "backroom", "circuit"         # Cluttered trajectory in the SOUS VIDE paper

#### 4. Simulate within the FiGS environment

In [ ]:
# Initialize the simulator and controller
sim = Simulator(scene_name,"eval_single","carl")
ctl = VehicleRateMPC("Viper",course_name,"carl")

# Use the ideal trajectory in VehicleRateMPC to get initial conditions and final time
t0,tf,x0 = ctl.tXUd[0,0],ctl.tXUd[-1,0],ctl.tXUd[0,1:11]

# Simulate the policy
Tro,Xro,Uro,Fro,Rgb,Dpt,Aux = sim.simulate(ctl,t0,tf,x0)

# Output the results as a video
gv.images_to_mp4(Rgb,course_name+'.mp4', ctl.hz)